In [2]:
import torch
import torchvision
import pandas as pd
import numpy as np
import sklearn
import matplotlib
from matplotlib import pyplot as plt
import time
import os
import re
os.getcwd()

'/Users/kangshuoli/Documents/VScode_workspace/GR5241/Proj_milestone_1'

In [3]:
# Data download and preprocessing
from torch.utils import data

DOWNLOAD_MNIST = False # If already download , set as False

train_data = torchvision.datasets.MNIST(
    root ="./mnist/",
    train = True , # this is training data
    # transform = torchvision.transforms.ToTensor(),
    download = DOWNLOAD_MNIST
)
test_data = torchvision.datasets.MNIST(root = "./mnist/", train = False)

# change the features to numpy
X_train = train_data.train_data.numpy()
X_test = test_data.test_data.numpy()

# change the labels to numpy
Y_train = train_data.train_labels.numpy()

Y_test = test_data.test_labels.numpy()

/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:54: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/Users/kangshuoli/miniforge3/envs/env_torch/lib/python3.9/site-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
# flatten the image
X_train_list = []
for i in np.arange(X_train.shape[0]):
    X_train_list.append(X_train[i,:,:].flatten(order = 'C'))
X_test_list = []
for i in np.arange(X_test.shape[0]):
    X_test_list.append(X_test[i,:,:].flatten(order = 'C'))
X_train_flattened = np.stack(X_train_list)
X_test_flattened = np.stack(X_test_list)

In [13]:
X_train_flattened_df = pd.DataFrame(data = X_train_flattened)
sample_index = X_train_flattened_df.sample(
    n = 6000, 
    replace = False, 
    random_state = 42
).index
X_train_subsampled = X_train_flattened_df.iloc[sample_index,:]
Y_train_subsampled = Y_train[sample_index]
assert X_train_subsampled.shape[0] == Y_train_subsampled.shape[0]
X_train_subsampled.index = np.arange(X_train_subsampled.shape[0])

In [15]:
params_svc = {
    'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5, 1], 
    'kernel': ['rbf'], 
    'degree': [3, 4, 5, 6], 
    'gamma': ['scale', 'auto'], 
    'shrinking': [True], 
    'tol': [0.0001, 0.00001], 
    'verbose': [False], 
    'random_state': [42]
}
# The best training score of SVM classifier is: 0.9532, the parameters used are: {'C': 1, 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'random_state': 42, 'shrinking': True, 'tol': 0.0001, 'verbose': False} The prediction score of SVM classifier is: 0.9583

gs_svc = GridSearchCV(
    estimator = SVC(), 
    param_grid = params_svc, 
    cv = 5, 
    scoring = 'accuracy', 
    refit = True, 
    n_jobs = -1, 
    verbose = 2
)

gs_svc.fit(X = X_train_subsampled, y = Y_train_subsampled)
print(f'The best training score of SVM classifier is: {gs_svc.best_score_:0.4f}, the parameters used are: {gs_svc.best_params_}')
print(f'The prediction score of SVM classifier is: {accuracy_score(Y_test, gs_svc.best_estimator_.predict(X_test_flattened)):0.4f}')

Fitting 5 folds for each of 128 candidates, totalling 640 fits
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=1e-05, verbose=False; total time=  39.9s
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=0.0001, verbose=False; total time=  39.9s
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=1e-05, verbose=False; total time=  40.0s
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=1e-05, verbose=False; total time=  40.0s
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=1e-05, verbose=False; total time=  40.1s
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=0.0001, verbose=False; total time=  40.0s
[CV] END C=0.001, degree=3, gamma=scale, kernel=rbf, random_state=42, shrinking=True, tol=0.0001, verbose=False; total time=  40.0s
[CV] END C=0.001,